The following code cell is building the sentence autoencoder using Keras.

In [ ]:
'''
variational autoencoder
Reference: "Auto-Encoding Variational Bayes" https://arxiv.org/abs/1312.6114
'''
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist

batch_size = 16
original_dim = 784  # sentence length
latent_dim = 2
intermediate_dim = 128  # encode size
epsilon_std = 0.01
nb_epoch = 40

x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_std = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_std = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., std=epsilon_std)
    return z_mean + K.exp(z_log_std) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
# so you could write `Lambda(sampling)([z_mean, z_log_std])`
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_std])

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

def vae_loss(x, x_decoded_mean):
    xent_loss = objectives.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_std - K.square(z_mean) - K.exp(z_log_std), axis=-1)
    return xent_loss + kl_loss

vae = Model(x, x_decoded_mean) 
vae.compile(optimizer='rmsprop', loss=vae_loss)

In [ ]:
vae.fit(_, _,
        shuffle=True,
        nb_epoch=nb_epoch,
        batch_size=batch_size,
        validation_data=(_, _),
        callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

# build a model to project inputs on the latent space
encoder = Model(x, z_mean)

The following code cell is building the sentence siamese network using Keras.

In [ ]:
from keras.engine.topology import Layer
from keras.layers import initializations
from keras import backend as K
import tensorflow as tf

# multi-sense vectors
class MultiSense(Layer):
    def __init__(self, num_sense, **kwargs):
        self.init = initializations.get('glorot_uniform')
        self.num_sense = num_sense
        
        super(MultiSense, self).__init__(**kwargs)

    def build(self, input_shape):
        # input_shape = R^{sentence_length x word_embedding}
        # for each word, we have (1,num_sense) weight
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        # TODO: to determine the input data size, we need to use batches
        # TODO: currently is the R^{num_sense x input_dim}, but desired R^{batch x num_sense x input_dim}
        self.W = self.init((self.num_sense, input_dim),
                             name='%s_W%d' % (self.name, i))
        self.trainable_weights = self.Ws

    def call(self, x, mask=None):
        # x.shape = input_shape
        # for each word, we elementwisely product with its weight
        # for each word, we keep the row that has the highest softmax score
        output = tf.mul(x, self.W)
        return output
        

    def get_output_shape_for(self, input_shape):
        return input_shape
    
    
from keras.models import Sequential
from keras.layers import LSTM

def create_shared_layer(output_dim, batch_input_dim):
    model = Sequential()
    model.add(MultiSense(60, input_shape=batch_input_dim))
    model.add(LSTM(output_dim))
    return model

shared_layer = create_shared_layer(128, (16, 784))

In [ ]:
from keras.models import Graph
from keras.layers import Lambda, Activation

batch_size = 16
original_dim = 784  # sentence length
intermediate_dim = 128  # encode size


g = Graph()
g.add_input(name='left', batch_input_shape=(batch_size, original_dim))
g.add_input(name='right', batch_input_shape=(batch_size, original_dim))

shared_layer = create_shared_layer(intermediate_dim, (batch_size, original_dim))
g.add_shared_node(shared_layer,
                  name='shared',
                  inputs=['left', 'right'],
                  merge_mode='cos',
                  create_output=False)
g.add_node(Activation('sigmoid'),
           name='d',
           input='shared')
g.add_output(name='output', 
             input='d')
g.compile(optimizer='RMSprop', loss={'output': 'mse'})

In [ ]:
shared_layer.predict()

The following code cell is the draft.

In [ ]:
import numpy as np
import theano.tensor as T
import theano.tensor.nnet as F
import theano


class HiddenLayer(object):
    def __init__(self, rng, ins, n_in, n_out, w=None, b=None, f=T.tanh):
        # initial weight and bias
        if w is not isinstance(w, np.ndarray):
            w = np.asarray(
                    rng.uniform(
                        low=-np.sqrt(6. / (n_in + n_out)),
                        high=np.sqrt(6. / (n_in + n_out)),
                        size=(n_in, n_out)
                    )
                )
            if f == theano.tensor.nnet.sigmoid:
                w *= 4
        if b is not isinstance(b, np.ndarray):
            b = np.zeros((n_out,))
       
        w = theano.shared(w, borrow=True)
        b = theano.shared(b, borrow=True)
        
        self.outs = T.dot(ins, w) - b
        if f is not None:
            # not a linear layer
            self.outs = f(self.outs)
        self.params = [w, b]
        

class NNet(object):
    def __init__(self, data, gstep=0.01, epochs=1000, rng=None):
        self.data = data
        self.ins = data
        self.n_sample, self.n_in = data.shape
        self.gstep = gstep   # gradient step
        self.layers = []
        self.params = []
        self.epochs = epochs
        if rng is None:
            self.rng = np.random
        else:
            self.rng = rng
        
    def add_layer(self, n_out, w=None, b=None, f=T.tanh):
        layer = HiddenLayer(rng=self.rng,
                            ins=self.ins,
                            n_in=self.n_in,
                            n_out=n_out,
                            w=w, b=b, f=f)
        self.layers.append(layer)
        # the output of this layer is the input of next layer
        self.ins = layer.outs
        self.n_in = n_out
        self.params += layer.params
        
    def train(self):
        x = T.dmatrix("x")
        # calculate the gradients
        gparams = T.grad(cost, [param for param in self.params])
        updates = [(param, param - self.gstep * gparam) for param, gparam in zip(self.params, gparams)]
        _train = theano.function(
                  inputs=[x],
                  outputs=self.cost,
                  updates=updates,
                  on_unused_input='ignore')
        for i in range(self.epochs):
            err = _train(self.data)
            print("Epoch: %d; Distance: %f" %(i+1, err))
        
    def set_cost(self, cost):
        self.cost = cost
    
    def get_final_outs(self):
        return self.layers[-1].outs
    

N = 5 # training sample size
IN_FEATS = 50 # input feature space
EPOCHS = 1000 # train iteration

rng = np.random
D = (rng.randn(N, IN_FEATS))      # inputs

nnet = NNet(D)
# encoder
nnet.add_layer(100)
# decoder
nnet.add_layer(IN_FEATS)
cost = ((nnet.get_final_outs() - D) ** 2).sum()
nnet.set_cost(cost)
nnet.train()

In [ ]:
# Autoencoder
import numpy as np
import theano.tensor as T
import theano.tensor.nnet as F
import theano

N = 5 # training sample size
IN_FEATS = 50 # input feature space, for encoder
OUT_FEATS = 100 # output feature space, for decoder
EPOCHS = 1000

rng = np.random
D = (rng.randn(N, IN_FEATS))      # inputs

x = T.dmatrix("x")

e_w1 = theano.shared(rng.randn(IN_FEATS, OUT_FEATS), name="encode_w1")  # weights
e_b1 = theano.shared(np.zeros(OUT_FEATS), name="encode_b1")             # bias
d_w1 = theano.shared(rng.randn(OUT_FEATS, IN_FEATS), name="decode_w1")
d_b1 = theano.shared(np.zeros(IN_FEATS), name="decode_b1")

encoder = F.sigmoid(T.dot(x, e_w1) - e_b1)
decoder = F.sigmoid(T.dot(encoder, d_w1) - d_b1)
cost = ((decoder - x) ** 2).sum()
e_gw1, e_gb1, d_gw1, d_gb1 = T.grad(cost, [e_w1, e_b1, d_w1, d_b1])

train = theano.function(
          inputs=[x],
          outputs=cost,
          updates=[(e_w1, e_w1 - 0.01*e_gw1), (e_b1, e_b1 - 0.01*e_gb1),
                   (d_w1, d_w1 - 0.01*d_gw1), (d_b1, d_b1 - 0.01*d_gb1)])
predict = theano.function(inputs=[x], outputs=encoder)

for i in range(EPOCHS):
    err = train(D)
    print("Epoch: %d; Distance: %f" %(i, err))

TODO: Psedocode of the CCA

```
func train_CCA(question_set, answer_set):
    matrix = build_cross_covariance_matrix(question_set, answer_set)
    U, s, V = svd(matrix)
    return U, V
```

```
func build_cross_covariance_matrix(question_set, answer_set):
    pair_num, M = question_set.shape
    pair_num, N = answer_set.shape
    indx = 0
    forEach question_set:
        avg = average(question_set[indx])
        question_set[indx] -= avg
        avg = average(answer_set[indx])
        answer_set[indx] -= avg
        i++
        
    initial matrix \in R^M x N fills by 0
    for i in range(M):
        for j in range(N):
            matrix[i, j] = sum(question_set[i] \dot answer_set[j].T) / (pair_num - 1)
    return matrix
```

```
func find_best_answer(question, answer_set, U, V):
    question_project = question \dot U
    initial best_answer as NULL
    initial best_similarity as INFINITY
    indx = 0
    forEach answer in answer_set:
        answer_project = answer \dot V
        similarity = cosine_distance(question_project, answer_project)
        if similarity <= best_similarity:
            best_similarity = similarity
            best_answer = indx
        indx++
    return indx
```

In [33]:
import numpy as np
from scipy.spatial.distance import cosine

q = np.random.randn(5, 4)
a = np.random.randn(5, 8)
    
def xcov(set1, set2):
    num_set1, M = set1.shape
    num_set2, N = set2.shape
    # one-to-one paired
    assert(num_set1 == num_set2)
    
    a = set1 - np.transpose([np.average(set1, axis=1)])
    b = set2 - np.transpose([np.average(set2, axis=1)])
    cov = np.dot(a.T, b) / (num_set1 - 1)
    
#     # cross-covariance matrix
#     cov = np.zeros((M, N), dtype='float64')
#     for i in range(num_set1):
#         for j in range(num_set1):
#             cov[i, j] = np.cov(m1[i],m2[j])[0][1]
            
    return cov

class CCA(object):
    def __init__(self):
        self.U = None
        self.V = None
    
    def train(self, q, a):
        '''
        params q: sentence embedding for question set
        params a: sentence embedding for answer set
        '''
        cov = xcov(q, a)
        self.U, s, self.V = np.linalg.svd(cov, full_matrices=False)
    
    def find_answer(self, v_q, a):
        assert self.U is not None and \
               self.V is not None
        v_q_proj = np.dot(v_q, self.U)
        
        best_distance = np.inf
        best_indx = 0
        for i, v_a in enumerate(a):
            v_a_proj = np.dot(v_a, self.V.T)
            s = cosine(v_q_proj, v_a_proj)
            if s <= best_distance:
                best_distance = s
                best_indx = i
        return best_indx

model = CCA()
model.train(q, a)
for v_q in q:
    print(model.find_answer(v_q, a))

3
4
1
4
1
